### **Script base: DAC+ADC**

Semplice script che funziona da oscilloscopio e controller del generatore di funzioni. L'oscilloscopio funziona a ciclo continuo e risponde a pochi comandi da tastiera:

* Tasto `'x'` esporta l'ultima misura su file (nome del file da indicare su prompt della shell python)
* Tasto `'space'` entra/esce da pausa nell'acquisizione
* Tasto `'esc'` termina il programma

In [9]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time

path = "/home/marco/Desktop/Uni_anno3/TD/Es_09/acquisizioni/"

# -[Configurazione Analog Discovery 2]-----------------------------------------
#   1. Connessiene con AD2
ad2 = tdwf.AD2()
ad2.vdd = 5
ad2.vss = 0
ad2.power(True)
#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.ampl = 2.5
wgen.w1.freq = 2*1270
wgen.w1.offs = 2.5
wgen.w1.func = tdwf.funcSquare
wgen.w1.duty = 50
wgen.w1.sync()
#wgen.w1.start()

wgen.w2.ampl = 2.5
wgen.w2.freq = 5
wgen.w2.offs = 2.5
wgen.w2.func = tdwf.funcSquare
wgen.w2.duty = 50
#wgen.w2.sync()
wgen.w2.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=1e5
scope.npt=10500
scope.ch1.rng = 50
scope.ch2.rng = 50
scope.ch1.avg=True
scope.ch2.avg=True
time.sleep(0.1)
scope.trig(True, level = 0.5, hist = 0.4, sour = tdwf.trigsrcCh1, cond=tdwf.trigslopeFall)



#   4. Configurazione powersupply

# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq
    if event.key == 'x':  # => export su file
        filename = input("Esporta dati su file: ")
        data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))
        if scope.npt > 8192:
            info =  f"Acquisizione Analog Discovery 2 - Lunga durata\ntime\tch1\tch2"
        else:
            info =  f"Acquisizione Analog Discovery 2\nTimestamp {scope.time.t0}\ntime\tch1\tch2"
        np.savetxt(filename, data, delimiter='\t', header=info)
    if event.key == ' ':  # => run/pausa misura
        flag_acq = not flag_acq
    if event.key == 'escape':  # => esci dalla misura
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
fig, ax = plt.subplots(figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True
while flag_run:
    if flag_acq: # l'acquisizione è attiva?
        #time.sleep(0.1)
        scope.sample()
    # Visualizzazione
    if flag_first:
        flag_first = False
        hp1, = plt.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        hp2, = plt.plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time [msec]", fontsize=15)
        plt.ylabel("Signal [V]", fontsize=15)
        plt.title("User interaction: x|space|escape")
        plt.tight_layout()
        plt.show(block = False)
    else:
        hp1.set_ydata(scope.ch1.vals)
        hp2.set_ydata(scope.ch2.vals)
        fig.canvas.draw()
        fig.canvas.flush_events()

plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321B5D8FC, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.
